# ASSIGNMENT: Cartoon Face Mask
## This assignment tests your ability to apply face recognition, image masks, image thresholding, video capturing and saving video feed into a video file concepts.

## TASK: Cartoonify faces in video feed from live webcam

### Steps
- 1. **Capture video** feed from webcam
- 2. **Recognize faces** in the video
- 3. **Replace/Mask the face** region with your favorite cartoon character
- 4. **Save the video** feed into a video file
- 5. Submit the following files
    - Notebook (.ipynb)
    - Video (.avi/.mp4)

In [1]:
!pip install opencv-python

In [2]:
import cv2
mask = cv2.imread("catmask.png", cv2.IMREAD_UNCHANGED)



In [3]:
import cv2
import os

# Load Haar cascades
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml")

# Load cartoon mask with alpha channel
mask = cv2.imread("catmask.png", cv2.IMREAD_UNCHANGED)

# Check if mask is loaded
if mask is None:
    print("Could not read mask image.")
    exit()

# Start webcam
video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    print("Could not open webcam.")
    exit()

# video
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# Automatically get path to Videos folder
video_path = os.path.join(os.environ['USERPROFILE'], 'Videos', 'output.avi')

out = cv2.VideoWriter(video_path, fourcc, 20.0, (640, 480))


while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Could not read frame from webcam.")
        break

    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eyeCascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)

        if len(eyes) >= 2:
            # Use first two detected eyes
            eye1 = eyes[0]
            eye2 = eyes[1]

            # Coordinates between both eyes
            x1 = min(eye1[0], eye2[0])
            y1 = min(eye1[1], eye2[1])
            x2 = max(eye1[0] + eye1[2], eye2[0] + eye2[2])
            y2 = max(eye1[1] + eye1[3], eye2[1] + eye2[3])

            mask_width = int((x2 - x1) * 2.2)
            mask_height = int((y2 - y1) * 2.5)

            center_x = x + x1 + (x2 - x1) // 2
            center_y = y + y1 + (y2 - y1) // 2

            x_offset = center_x - mask_width // 2
            y_offset = center_y - mask_height // 2

            # Resize mask
            mask_resized = cv2.resize(mask, (mask_width, mask_height))
            mask_rgb = mask_resized[:, :, :3]
            mask_alpha = mask_resized[:, :, 3] / 255.0

            x1f, x2f = x_offset, x_offset + mask_width
            y1f, y2f = y_offset, y_offset + mask_height

            if x1f < 0 or y1f < 0 or x2f > frame.shape[1] or y2f > frame.shape[0]:
                continue

            roi = frame[y1f:y2f, x1f:x2f]
            for c in range(3):
                roi[:, :, c] = roi[:, :, c] * (1 - mask_alpha) + mask_rgb[:, :, c] * mask_alpha
            frame[y1f:y2f, x1f:x2f] = roi

    cv2.imshow("Cartoon Mask", frame)
    out.write(frame)

    # exit
    key = cv2.waitKey(1)
    if key == ord('q') or cv2.getWindowProperty("Cartoon Mask", cv2.WND_PROP_VISIBLE) < 1:
        break

# Cleanup
video_capture.release()
out.release()
cv2.destroyAllWindows()



### Helper code to recognize faces